In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2', pad_token_id=tokenizer.eos_token_id)


In [2]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

# Load the dataset
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='scifi.txt',
    block_size=128
)

/home/xiayuu/.local/lib/python3.9/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [3]:
import torch
# Split the dataset into training and validation subsets
train_size = int(len(dataset) * 0.9)  # Use 90% of the data for training
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
print(train_size, val_size)
# Define the data collator for language modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


243564 27063


In [4]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    num_train_epochs=3,
    per_device_train_batch_size=50,
    per_device_eval_batch_size=50,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=1000,
    save_total_limit=1,
)

# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator
)

In [5]:
import math
eval_loss = trainer.evaluate(val_dataset)['eval_loss']
perplexity = math.exp(eval_loss)
print(f'Perplexity without Finetuning: {perplexity:.2f}')

Perplexity without Finetuning: 62.86


In [6]:
# Train the model
trainer.train()

# Evaluate the perplexity on the test set
eval_loss = trainer.evaluate(val_dataset)['eval_loss']
perplexity = math.exp(eval_loss)
print(f'Perplexity after Finetuning: {perplexity:.2f}')

/home/xiayuu/.local/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.906300,3.801984
2,3.810000,3.759054
3,3.766500,3.746405


Perplexity after Finetuning: 42.37


In [29]:
prompt_1 = "The year is 2050 and humanity has just discovered a new planet capable of sustaining life. A team of scientists and explorers are sent to investigate. What they find will change the course of human history forever."
prompt_2 = "In a world where technology has advanced to the point of being able to upload human consciousness into a virtual reality, one man must navigate this new world and the dangers that come with it. But when he discovers a dark secret about the virtual world, he must decide whether to stay in blissful ignorance or risk everything to expose the truth."
prompt_3 = "After a catastrophic event leaves Earth uninhabitable, the last remaining humans are forced to live on a space station orbiting the planet. But when a group of rebels discover a way to potentially save Earth, they must decide whether to risk their lives for a chance at redemption or continue living in their artificial home."

generated_text = model.generate(
    input_ids = torch.tensor(tokenizer.encode(prompt_3)).unsqueeze(0).to('cuda'),
    max_length=300,
    temperature=0.5,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2,
    num_return_sequences=1,
)

print(tokenizer.decode(generated_text[0], skip_special_tokens=True))


After a catastrophic event leaves Earth uninhabitable, the last remaining humans are forced to live on a space station orbiting the planet. But when a group of rebels discover a way to potentially save Earth, they must decide whether to risk their lives for a chance at redemption or continue living in their artificial home. The book is one of those rare gems that have never been published before -- and it's an important addition to any science fiction library! (Publ., #) by Robert Silverberg  Theodore Sturgeon Isaac Asimov Harlan Ellison Damon Knight Frank Herbert Locus Press Edmond Hamilton Fritz Leiber Jack Vance Lester del Rey Frederik Pohl James Blish Roger Zelazny Larry Niven A S E J H OW was born into slavery during World War II. He came from a family of wealthy farmers who had bought up all the land under the New York Stock Exchange as slaves; but his parents were able only to sell them out because he could not find enough money to buy back what he needed. When he reached colleg

In [9]:
trainer.save_model('scifigpt')